In [1]:
import typing as t
import operator as op
import subprocess as sp
import glob
import os
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from itertools import chain

import tqdm
import pandas as pd
from fn import F
from Bio import SearchIO, SeqIO

pd.set_option('display.max_rows', 1000)
%matplotlib inline

Parse hmmscan output and extract domain annotations

In [2]:
scan_files = [
    'search/hmmscan/sp.txt',
    'search/hmmscan/chz.txt'
]
samples = ['sp', 'chz']

scans = [
    list(SearchIO.parse(fname, 'hmmer3-text')) for fname in scan_files
]


In [3]:
hmmscan_annotation_keys = [
    'query', 'model', 'description', 'bitscore', 
    'evalue', 'domains_expected', 'domains_observed'
]

def extract_query_annotations(query) -> pd.DataFrame:
    query_id = query.id
    hits = query.hits
    hit_ids = [hit.id for hit in hits]
    hit_descriptions = [hit.description for hit in hits]
    hit_scores = [hit.bitscore for hit in hits]
    hit_evalues = [hit.evalue for hit in hits]
    domains_expected = [hit.domain_exp_num for hit in hits]
    domains_observed = [hit.domain_obs_num for hit in hits]
    records = [
        [query_id, hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs]
        for hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs, in 
        zip(hit_ids, hit_descriptions, hit_scores, hit_evalues,
            domains_expected, domains_observed)
    ]
    return pd.DataFrame(records, columns=hmmscan_annotation_keys)


def extract_hmmscan_annotations(sample_id, sample_queries) -> pd.DataFrame:
    empty = pd.DataFrame(
        data=[['NA', 'NA', 'NA', 0, 99999.0, 0.0, 0.0]], 
        columns=hmmscan_annotation_keys
    )
    hmmscan = (
        F(map, extract_query_annotations)
        >> list
        >> (lambda dfs: pd.concat(dfs) if dfs else empty)
        >> (lambda df: df.reset_index(drop=True))
    )(sample_queries)
    return pd.concat([
        pd.Series(sample_id, index=hmmscan.index, name='sample'),
        hmmscan
    ], axis=1)


In [4]:
scan_annotations = [
    extract_hmmscan_annotations(id_, scan) for id_, scan in zip(samples, scans)
]


In [5]:
scan_annotations_concat = (
    pd.concat(scan_annotations)
    .reset_index(drop=True)
    .sort_values(['sample', 'description'])
)
scan_annotations_concat.head()


,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
81,chz,NBJEAOJI_45955,PF00150.18,Cellulase (glycosyl hydrolase family 5),86.0,3.400000e-27,1.1,1
98,chz,NBJEAOJI_60505,PF00150.18,Cellulase (glycosyl hydrolase family 5),102.0,4.300000e-32,1.3,1
120,chz,NBJEAOJI_86854,PF00150.18,Cellulase (glycosyl hydrolase family 5),100.4,1.400000e-31,1.1,1
121,chz,NBJEAOJI_88210,PF00150.18,Cellulase (glycosyl hydrolase family 5),76.8,2.100000e-24,1.0,1
122,chz,NBJEAOJI_88582,PF00150.18,Cellulase (glycosyl hydrolase family 5),64.2,1.500000e-20,1.0,1


In [7]:
# ambiguity check: several domain profiles matching the same query?
scan_annotations_concat.groupby('query').apply(len).max()


1

In [6]:
os.makedirs('search/glyco', exist_ok=True)


In [8]:
# export domain annotations
scan_annotations_concat.to_csv('search/glyco/annotations.tsv', sep='\t', index=False)


Extract nucleotide (or protein) sequences with positive matches

In [ ]:
scan_annotations_concat['description'].value_counts()


In [ ]:
selected_queries = set(scan_annotations_concat['query'])
proteins = [
    SeqIO.parse('annotation/sp/PROKKA_01112020.ffn', 'fasta'),
    SeqIO.parse('annotation/chz/PROKKA_12302019.ffn', 'fasta')
]
for samp, prots in zip(samples, proteins):
    SeqIO.write((seq for seq in prots if seq.id in selected_queries), f'search/glyco/{samp}.faa', 'fasta')